In [1]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
url = "https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD"
df = pd.read_csv(url)

In [3]:
df.shape

(397660, 26)

In [15]:
name = "Udon West".strip().upper()
boro = "Manhattan".strip().title()
street = "".strip().upper()
zipcode = "".strip()

In [16]:
name = name if name else None
boro = boro if boro else None
zipcode = zipcode if zipcode else None

In [17]:
if boro and boro not in ['Manhattan', 'Queens', 'Brooklyn', 'Staten Island', 'Bronx']:
    raise ValueError(f"Unknown borough {boro}")

if zipcode and zipcode not in [int(z) for z in df.ZIPCODE.unique() if np.isfinite(z)]:
    raise ValueError(f"Unknown ZIP code {zipcode}")

In [18]:
query_strs = " & ".join([
    f"{column_name} == '{field}'" for column_name, field in [("DBA", name), ("BORO", boro), ("STREET", street)] if field
])

query_nums = " & ".join([
    f"{column_name} == '{field}'" for column_name, field in [("ZIPCODE", zipcode)] if field
])

query = " & ".join([q for q in [query_strs, query_nums] if q])

In [99]:
def sanitize(s, pad=None):
    if not isinstance(s, str):
        s = str(s)

    return " ".join(s.title().split())

In [20]:
filtered = df.query(query)
num_results = filtered["CAMIS"].nunique()

In [100]:
choices = filtered[["CAMIS", "DBA", "BUILDING", "STREET", "BORO", "CUISINE DESCRIPTION"]].drop_duplicates()
lengths = choices.applymap(lambda s: len(sanitize(s))).max()

for _, row in choices.iterrows():
    print(
        f"*{sanitize(row['DBA'])}:* "
        f"{sanitize(row['CUISINE DESCRIPTION'])}, "
        f"{sanitize(row['BUILDING'])} "
        f"{sanitize(row['STREET'])}, "
        f"{sanitize(row['BORO'])}"
    )

*Udon West:* Japanese, 150 East 46 Street, Manhattan
*Udon West:* Japanese, 11 St Marks Place, Manhattan


In [107]:
filtered = filtered[filtered.CAMIS == 41380163]

In [109]:
filtered.columns

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE', 'Latitude', 'Longitude',
       'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL',
       'NTA'],
      dtype='object')

In [203]:
violations = filtered[
    ["INSPECTION DATE", "ACTION", "VIOLATION CODE", "VIOLATION DESCRIPTION", 'CRITICAL FLAG', 'GRADE', 'GRADE DATE']
]
violations.loc[:, "INSPECTION DATE"] = pd.to_datetime(violations.loc[:, "INSPECTION DATE"])
violations = violations.sort_values("INSPECTION DATE", ascending=False)

In [218]:
def interpret_action(s):
    if "establishment closed" in s.lower():
        return "we were closed"
    elif "establishment re-opened" in s.lower():
        return "we were allowed to re-open"
    elif "establishment re-closed" in s.lower():
        return "we were re-closed"
    elif "violations were cited" in s.lower():
        return "we were cited for the following violation(s):"
    elif "no violations" in s.lower():
        return "no violations were found."
    else:
        return "no violations were found."

In [219]:
def truncate_violation_description(s):
    return s.split(".")[0] + "."

In [269]:
violations.loc[v, ["VIOLATION DESCRIPTION", "CRITICAL FLAG"]]

,VIOLATION DESCRIPTION,CRITICAL FLAG
250681,Filth flies or food/refuse/sewage-associated (...,Y
316194,Facility not vermin proof. Harborage or condit...,N


In [272]:
for _, v in sorted(violations.groupby(["INSPECTION DATE", "ACTION"]).groups.items(), reverse=True):
    row = violations.loc[v].iloc[0]  # Just get one row
    print(
        f"On {row['INSPECTION DATE'].strftime('%Y-%m-%d')}, "
        f"{interpret_action(row['ACTION'])}"
    )
    
    for i, (description, critical) in violations.loc[v, ["VIOLATION DESCRIPTION", "CRITICAL FLAG"]].iterrows():
        if critical == "Y":
            print(f"  - *{truncate_violation_description(description)}*")            
        else:
            print(f"  - {truncate_violation_description(description)}")

On 2019-06-17, we were cited for the following violation(s):
  - Non-food contact surface improperly constructed.
  - *Cold food item held above 41º F (smoked fish and reduced oxygen packaged foods above 38 ºF) except during necessary preparation.*
On 2018-10-01, we were cited for the following violation(s):
  - Facility not vermin proof.
  - Plumbing not properly installed or maintained; anti-siphonage or backflow prevention device not provided where required; equipment or floor not properly drained; sewage disposal system in disrepair or not functioning properly.
  - *Filth flies or food/refuse/sewage-associated (FRSA) flies present in facility’s food and/or non-food areas.*
On 2018-09-17, we were cited for the following violation(s):
  - Facility not vermin proof.
  - *Live roaches present in facility's food and/or non-food areas.*
  - Plumbing not properly installed or maintained; anti-siphonage or backflow prevention device not provided where required; equipment or floor not prope

In [220]:
for _, row in violations.iterrows():
    print(
        f"On {row['INSPECTION DATE'].strftime('%Y-%m-%d')}, "
        f"{interpret_action(row['ACTION'])}\n\t"
        f"{truncate_violation_description(row['VIOLATION DESCRIPTION'])}"
    )

On 2019-06-17, we were cited for the following violation(s):
	Non-food contact surface improperly constructed.
On 2019-06-17, we were cited for the following violation(s):
	Cold food item held above 41º F (smoked fish and reduced oxygen packaged foods above 38 ºF) except during necessary preparation.
On 2018-10-01, we were cited for the following violation(s):
	Facility not vermin proof.
On 2018-10-01, we were cited for the following violation(s):
	Plumbing not properly installed or maintained; anti-siphonage or backflow prevention device not provided where required; equipment or floor not properly drained; sewage disposal system in disrepair or not functioning properly.
On 2018-10-01, we were cited for the following violation(s):
	Filth flies or food/refuse/sewage-associated (FRSA) flies present in facility’s food and/or non-food areas.
On 2018-09-17, we were cited for the following violation(s):
	Facility not vermin proof.
On 2018-09-17, we were cited for the following violation(s):
